In [1]:
import os
import logging
import requests
from trello import TrelloClient
from datetime import datetime
from telegram import File
from telegram import Update, ReplyKeyboardMarkup, ReplyKeyboardRemove
from telegram.ext import (
    Updater, CommandHandler, MessageHandler, Filters, ConversationHandler, CallbackContext
)
from telegram import KeyboardButton # Для создания кнопок на клавиатуре в чатах с ботами

# Токен от BotFather
TOKEN = "BOT_TOKEN"

# Создаем экземпляр клиента Trello с использованием API ключа
TRELLO_API_KEY = "API_TRELLO"
TRELLO_API_TOKEN = "TOKEN_TRELLO"
client = TrelloClient(
    api_key=TRELLO_API_KEY,
    api_secret='SECRET_API',
    token=TRELLO_API_TOKEN,
    token_secret='SECRET_TOKEN'
)

# Уровень логирования
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

# Состояния для конечного автомата (этапность)
CHOOSING, CONTACT_INFO, GETTING_EMAIL, FILLING_FIO, FILLING_STUFF, REPORT_TYPE, REPORT_NUMBER, DESCRIBE_ISSUE, \
LVL_REQUEST, ATTACHING_FILE, FINISHING_PROCESS = range(11)

def start(update: Update, context: CallbackContext) -> int:
    logger.info("Пользователь %s начал диалог", update.message.from_user.username)
    update.message.reply_text(
        "Добро пожаловать! Начнем. Пожалуйста, предоставьте номер телефона ответственного исполнителя"
    )
    return CHOOSING
    
def contact_received(update: Update, context: CallbackContext) -> int:
    user_data = context.user_data
    user_data['contact'] = update.message.text
    update.message.reply_text(
        "Выберите тип заявки:",
        reply_markup=ReplyKeyboardMarkup([['Поддержка', 'ЗНИ']], one_time_keyboard=True)
    )
    return CONTACT_INFO

def getting_email(update: Update, context: CallbackContext) -> int:
    user_data = context.user_data
    user_data['email'] = update.message.text
    update.message.reply_text(
        "Спасибо! Теперь введите Почту ответственного исполнителя:"
    )
    return GETTING_EMAIL

def type_selected(update: Update, context: CallbackContext) -> int:
    user_data = context.user_data
    user_data['email'] = update.message.text
    logger.debug("Email collected: %s", user_data['email'])
    update.message.reply_text(
        "Введите ваше ФИО:"
    )
    return FILLING_FIO

def fio_received(update: Update, context: CallbackContext) -> int:
    user_data = context.user_data
    user_data['fio'] = update.message.text
    update.message.reply_text(
        "Введите вашу должность:"
    )
    return FILLING_STUFF

def type_selected_1(update: Update, context: CallbackContext) -> int:
    user_data = context.user_data
    user_data['request_type'] = update.message.text
    update.message.reply_text(
        "Выберите тип отчета:",
        reply_markup=ReplyKeyboardMarkup([['Регуляторная', 'Управленческая', 'Дэшборд']], one_time_keyboard=True)
    )
    return REPORT_TYPE

# Ваш обработчик выбора номера отчета
def report_type_selected(update: Update, context: CallbackContext) -> int:
    user_data = context.user_data
    user_data['report_type'] = update.message.text
    update.message.reply_text(
        "Напишите номер и название отчета"
    )
    return REPORT_NUMBER  # Переход к обработке ввода описания проблемы

def report_number_received(update: Update, context: CallbackContext) -> int:
    user_data = context.user_data
    user_data['report_number'] = update.message.text
    update.message.reply_text(
        "Введите описание проблемы:"
    )
    return DESCRIBE_ISSUE

def description_received(update: Update, context: CallbackContext) -> int:
    user_data = context.user_data
    user_data['description'] = update.message.text
    update.message.reply_text(
        "Выберите уровень срочности:",
        reply_markup=ReplyKeyboardMarkup([['Критичный', 'Некритичный']], one_time_keyboard=True)
    )
    return LVL_REQUEST

def urgency_selected(update: Update, context: CallbackContext) -> int:
    user_data = context.user_data
    user_data['urgency'] = update.message.text

    update.message.reply_text(
        "Пожалуйста, прикрепите файлы в котором имеются эталонные значения в формате Excel, CSV или TXT. \nИ если имеется скрин, то добавьте ее в этот же файл"
    )
    return ATTACHING_FILE

def file_attached(update: Update, context: CallbackContext) -> int:
    user_data = context.user_data

    if 'attached_files' not in user_data:
        user_data['attached_files'] = []

    attached_file = update.message.document
    
    if attached_file is not None:
        # Записываем детали вложения
        logger.debug("Attached File: %s, Mime Type: %s", attached_file.file_name, attached_file.mime_type)

        accepted_mime_types = [
            "application/vnd.ms-excel",  # Excel в старом формате (XLS)
            "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",  # Excel в новом формате (XLSX)
            "text/plain",                # TXT
            "text/csv"                   # CSV
        ]

        if attached_file.mime_type in accepted_mime_types:
            if 'file' not in user_data:
                user_data['file'] = []

            user_data['file'].append(attached_file.get_file())
            
            # Добавляем прикрепленный файл в список attached_files
            user_data['attached_files'].append(attached_file)
            
            update.message.reply_text(
                    "Файл успешно добавлен! Вы можете добавить еще файл или завершить процесс.",
                    reply_markup=ReplyKeyboardMarkup([['Добавить еще файл', 'Завершить процесс\n Есть же!']], one_time_keyboard=True)
                )
            return ATTACHING_FILE
        else:
            message_text = "Я есть ГРУТ! Прикрепляй давай!."
    else:
        message_text = "Я есть ГРУТ! Прикрепляй давай!."

    update.message.reply_text(
        message_text,
        reply_markup=ReplyKeyboardRemove()
    )
    return FINISHING_PROCESS

def finishing_process(update: Update, context: CallbackContext) -> int:
    # Процесс завершения здесь 
    save_request(update, context)  # Вызов функции save_request
    return ConversationHandler.END
    
def save_request(update: Update, context: CallbackContext):
    user_data = context.user_data  # Получаем данные пользователя
    message = update.message  # Получаем сообщение пользователя
    
    logger.debug("User data: %s", user_data)
    
    request_info = (
        f"Контактные данные: {user_data['contact']}\n"
        f"ФИО инициатора: {user_data['fio']}\n"
        f"Почта: {user_data['email'].encode('utf-8').decode('utf-8')}\n"
        f"Тип заявки: {user_data['report_type']}\n"
        f"Описание проблемы: {user_data['description']}\n"
        f"Уровень срочности: {user_data['urgency']}\n"
    )
    
    logger.debug("Email used in request_info: %s", user_data['email'])
    
    current_date = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    folder_path = f"D:/saved_requests/{current_date}"
    os.makedirs(folder_path)

    with open(os.path.join(folder_path, "request_info.txt"), "w", encoding="utf-8") as file:
        file.write(request_info)

    # Процесс прикрепления файла
    if 'attached_files' in user_data and user_data['attached_files']:
        for idx, attached_file in enumerate(user_data['attached_files']):
            file_extension = attached_file.file_name.split('.')[-1]
            file_name = f"attachment_{idx}.{file_extension}"
            file_path = os.path.join(folder_path, file_name)
            attached_file.get_file().download(file_path)
            logger.info("Attached File %d saved: %s", idx, file_name)
        
    logger.info("Заявка сохранена: %s", request_info)
    
    # Отправляем сообщение о успешном сохранении заявки
    message.reply_text(
        "Спасибо за обращение!\nЧао-Какао\nИ если есть еще обращения нажми сюда /start",
        reply_markup=ReplyKeyboardRemove()
    )
    
    # Создаем заявки на доске Trello
    board_id = 'BOARD_ID_TRELLO'  # Заменяем на ID доски если необходимо
    board = client.get_board(board_id)
    list_id = 'LIST_ID_TRELLO'  # Заменяем на ID списка на доске если необходимо
    list_ = board.get_list(list_id)
    
    
    # Создаем карточку на Trello и добавляем пользовательские поля
    if 'attached_files' in user_data and user_data['attached_files']:
        # Создаем карточку в Trello и получаем ее идентификатор
        current_datetime = datetime.strptime(current_date, "%Y-%m-%d_%H-%M-%S") # Convert the current date string to a datetime object
        request_id = current_datetime.strftime("%Y%m%d%H%M%S")  # Форматируем дату и время
        card = list_.add_card(
            name=f"Заявка #{request_id} от {user_data['contact']} ({user_data['request_type']})",
            desc=f"Описание проблемы: {user_data['description']}\n",
            due=None)  # Указываем срок выполнения, если необходимо
        
        # Добавляем дополнительные поля к карточке
        custom_fields = {
            "Контактные данные": user_data['contact'],
            "Почта": user_data['email'].encode('utf-8').decode('utf-8'),
            "ФИО инициатора": user_data['fio'],
            "Тип заявки": user_data['report_type'],
            "Уровень срочности": user_data['urgency'],
        }

        # Добавляем данные пользовательского поля в описание карточки
        card_description = card.desc + "\nКраткий обзор:\n" + "\n".join([f"{field}: {value}" for field, value in custom_fields.items()])
        card.set_description(card_description)
        
        # Прикрепляем файлы к карточке Trello
        for idx, attached_file in enumerate(user_data['attached_files']):
            file_extension = attached_file.file_name.split('.')[-1]
            file_name = f"attachment_{idx}.{file_extension}"
            file_path = os.path.join(folder_path, file_name)
            
            # Скачиваем файл с помощью метода get_file объекта File
            file = attached_file.get_file()
            file.download(file_path)

            # URL-адрес API Trello для прикрепления файла к карточке
            attachments_url = f"https://api.trello.com/1/cards/{card.id}/attachments"

            # Параметры API Trello
            params = {
                'key': TRELLO_API_KEY,
                'token': TRELLO_API_TOKEN,
                'name': file_name
            }

            # Открываем и отправляем файл в Trello
            with open(file_path, 'rb') as file:
                response = requests.post(attachments_url, params=params, files={'file': file})

            if response.status_code == 200:
                logger.info("Файл прикреплен к карточке Trello: %s", file_name)
            else:
                logger.warning("Не удалось прикрепить файл к карточке Trello. Status code: %s, Response: %s", response.status_code, response.text)
    
def main():
    updater = Updater(token=TOKEN, use_context=True)
    dispatcher = updater.dispatcher

    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],
        states={
            CHOOSING: [MessageHandler(Filters.text & ~Filters.command, contact_received)],
            CONTACT_INFO: [MessageHandler(Filters.regex('^(Поддержка|ЗНИ)$'), getting_email)],
            GETTING_EMAIL: [MessageHandler(Filters.text & ~Filters.command, type_selected)],
            FILLING_FIO: [MessageHandler(Filters.text & ~Filters.command, fio_received)],
            FILLING_STUFF: [MessageHandler(Filters.text & ~Filters.command, type_selected_1)],
            REPORT_TYPE: [MessageHandler(Filters.regex('^(Регуляторная|Управленческая|Дэшборд)$'), report_type_selected)],
            REPORT_NUMBER: [MessageHandler(Filters.text & ~Filters.command, report_number_received)],
            DESCRIBE_ISSUE: [MessageHandler(Filters.text & ~Filters.command, description_received)],
            LVL_REQUEST: [MessageHandler(Filters.text & ~Filters.command, urgency_selected)],
            ATTACHING_FILE: [
                # Обработчик для прикрепленных файлов с определенными MIME-типами
                MessageHandler(
                    Filters.document.mime_type("application/vnd.ms-excel") |
                    Filters.document.mime_type("application/vnd.openxmlformats-officedocument.spreadsheetml.sheet") |
                    Filters.document.mime_type("text/csv") |
                    Filters.document.mime_type("text/plain"),
                    file_attached
                ),
                MessageHandler(Filters.regex('^Добавить еще файл$'), file_attached),
                MessageHandler(Filters.regex('^Я есть ГРУТ! Прикрепляй давай!$'), save_request),
                MessageHandler(Filters.regex('^Завершить процесс\n Есть же!$'), save_request),
            ],
            FINISHING_PROCESS: [
                MessageHandler(Filters.regex('^Завершить процесс\n Есть же!$'), finishing_process),
                MessageHandler(Filters.regex('^Завершить процесс\n Есть же!$'), save_request),
                MessageHandler(Filters.regex('^Я есть ГРУТ! Прикрепляй давай!$'), save_request)
            ],
        },
        fallbacks=[]
    )

    # Добавляем ConversationHandler в диспетчер
    dispatcher.add_handler(conv_handler)

    # Создаем обработчик для прикрепленных файлов, который будет реагировать на разные типы файлов
    file_handler = MessageHandler(
        Filters.document.mime_type("application/vnd.ms-excel") |
        Filters.document.mime_type("application/vnd.openxmlformats-officedocument.spreadsheetml.sheet") |
        Filters.document.mime_type("text/csv") |
        Filters.document.mime_type("text/plain"),
        file_attached
    )

    # Добавляем обработчик прикрепленных файлов в диспетчер
    dispatcher.add_handler(file_handler)

    # Запускаем бота, чтобы он начал получать обновления от Telegram
    updater.start_polling()

    # Оставляем бота активным, чтобы он продолжал работать
    updater.idle()

# Запускаем функцию main, только если скрипт выполняется напрямую (не импортирован как модуль)
if __name__ == '__main__':
    main()

InvalidToken: Invalid token